In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))


In [2]:
def get_weekly_stress_score(week_context_attributes):
    """
    Return stress score for the week
    :param week_context_attributes: How days context looks like
    :return: weekly stress score based on FSM
    """
    stress_score = 0.
    total_days_data_in_week = len(week_context_attributes.keys())

    # Condition 1: Context of exercising and housework existing in 57% of week: -1 stress score
    c1_exercise_count, c1_housework_count = 0.,0.
    for day in week_context_attributes.keys():
        if 'Exercising' in week_context_attributes[day].context.values:
            c1_exercise_count +=1
        if 'HouseWork' in week_context_attributes[day].context.values:
            c1_housework_count+=1

    if ((c1_exercise_count/total_days_data_in_week) >= 0.57) and ((c1_housework_count/total_days_data_in_week) >= 0.57):
        stress_score -= 1

    # Condition 2a: No context of exercising for more than 57% week(4 days if full week available): +0.5 stress score
    # Condition 2b: No context of housework  for more than 4 days(57% week): +0.5 stress score
    c2a_exercise_count = c1_exercise_count
    c2b_housework_count = c1_housework_count

    if (c2a_exercise_count/total_days_data_in_week) < 0.57:
        stress_score += 0.5
    if (c2b_housework_count/total_days_data_in_week) < 0.57:
        stress_score += 0.5

    # Condition 3: context of commute for more than an hour(on average) in a day everyday: +0.5 stress score
    c3_commute_hours = []
    for day in week_context_attributes:
        if 'Commuting' not in week_context_attributes[day].context.values:
            c3_commute_hours.append(0.)
        else:
            day_commute_values = week_context_attributes[day][week_context_attributes[day].context=='Commuting']
            day_commute_values['period'] = day_commute_values['end'] - day_commute_values['start']
            c3_commute_hours.append(day_commute_values['period'].sum())

    if np.mean(c3_commute_hours) > (60*60): # more than 1 hours of average commute
        stress_score +=0.5


    # condition 4a: context of sleep for less than 6 hours on more than 3 days(42% week): +1 stress score
    # condition 4b: context of sleep for less more than 8 hours on an average: -0.5 stress score
    c4_sleep_hours = []
    for day in week_context_attributes:
        if 'Sleeping' not in week_context_attributes[day].context.values:
            c4_sleep_hours.append(0.)
        else:
            day_sleep_values = week_context_attributes[day][week_context_attributes[day].context == 'Sleeping']
            day_sleep_values['period'] = day_sleep_values['end'] - day_sleep_values['start']
            c4_sleep_hours.append(day_sleep_values['period'].sum())

    if np.max(sorted(c4_sleep_hours)[:3]) < (6 * 60 * 60):  # 4a.less than 6 hours of sleep for 3 days
        stress_score += 1

    if np.mean(c4_sleep_hours) > (8 * 60 * 60):  # 4b. more than 8 hours of sleep
        stress_score -= 0.5

    # condition 5: (More than 4 days(57%) of Exercising context for more than half an hour) &
    # (Different working hours (>40% difference) across days): +0.5 stress score
    c5_exercising_count = c1_exercise_count
    c5_exercising_hours = []
    for day in week_context_attributes:
        if 'Exercising' in week_context_attributes[day].context.values:
            day_exercise_values = week_context_attributes[day][week_context_attributes[day].context == 'Exercising']
            day_exercise_values['period'] = day_exercise_values['end'] - day_exercise_values['start']
            c5_exercising_hours.append(day_exercise_values['period'].sum())
    c5_exercise_cond = False
    if ((c5_exercising_count/total_days_data_in_week) >= 0.57) and (np.min(c5_exercising_hours) > (0.5*60*60)):
        c5_exercise_cond = True

    c5_working_periods = {}
    for day in week_context_attributes:
        if 'OfficeWork' in week_context_attributes[day].context.values:
            day_work_values = week_context_attributes[day][week_context_attributes[day].context == 'OfficeWork']
            day_work_values['start_hour'] = pd.to_datetime(day_work_values['start'],unit='s').dt.hour
            day_work_values['end_hour'] = pd.to_datetime(day_work_values['end'], unit='s').dt.hour
            c5_working_periods[day] = np.zeros(24)
            for idx, row in day_work_values.iterrows():
                c5_working_periods[day][row['start_hour']:row['end_hour']+1] = 1

    c5_hour_differences= [0.]
    for dayA in c5_working_periods.keys():
        for dayB in c5_working_periods.keys():
            if not (dayA==dayB):
                working_hoursA = c5_working_periods[dayA]
                working_hoursB = c5_working_periods[dayB]
                diff_hours = np.sum(working_hoursA!=working_hoursB)
                same_hours = np.sum(np.logical_and(working_hoursA==working_hoursB,working_hoursA))
                if same_hours > 0.:
                    c5_hour_differences.append(diff_hours / same_hours)
                else:
                    c5_hour_differences.append(0.)

    c5_working_cond = False
    if np.max(c5_hour_differences) > 0.4:
        c5_working_cond = True

    if (c5_working_cond) & (c5_exercise_cond):
        stress_score +=0.5

    # condition 6: (No context of exercising) & (Different working hours across days): +1 stress score
    c6_exercise_count=  c1_exercise_count
    c6_working_hour_diff = c5_hour_differences
    if (c6_exercise_count==0.) & (np.mean(c5_hour_differences) > 0.4):
        stress_score += 1

    # condition 7: (More than 4 days of Exercising context for more than half an hour) &
    # (No Different working hours (>40% difference)  across days): -1 stress score
    c7_exercise_cond = c5_exercise_cond
    c7_hour_differences = c5_hour_differences

    if (c7_exercise_cond) & (np.max(c7_hour_differences) < 0.4):
        stress_score -= 1.

    # condition 8: (More than 10 hrs of works in any day) OR (50+ hours per week): +1 stress score
    c8_work_hours = []
    for day in week_context_attributes:
        if 'OfficeWork' in week_context_attributes[day].context.values:
            day_work_values = week_context_attributes[day][week_context_attributes[day].context == 'OfficeWork']
            day_work_values['period'] = day_work_values['end'] - day_work_values['start']
            c8_work_hours.append(day_work_values['period'].sum())
        else:
            c8_work_hours.append(0.)

    if (np.max(c8_work_hours) > (10 * 60 * 60)) or (np.sum(c8_work_hours) > (50 * 60 * 60)):
        stress_score += 1

    # condition 9a: (More than 40% shift in working hours in max gap): +1 stress score
    # condition 9b: (More than 40% shift in working hours in 2 or more consecutive days): +1 stress score
    # Not modeling due to over complexity

    # condition 10a: More than 15 mins exercising every day: -0.5 stress score
    # condition 10b: More than 1 hour housework every day: -0.5 stress score
    c10a_exercising_hours = []
    for day in week_context_attributes:
        if 'Exercising' in week_context_attributes[day].context.values:
            day_exercise_values = week_context_attributes[day][week_context_attributes[day].context == 'Exercising']
            day_exercise_values['period'] = day_exercise_values['end'] - day_exercise_values['start']
            c10a_exercising_hours.append(day_exercise_values['period'].sum())
        else:
            c10a_exercising_hours.append(0.)

    c10b_housework_hours = []
    for day in week_context_attributes:
        if 'HouseWork' not in week_context_attributes[day].context.values:
            c10b_housework_hours.append(0.)
        else:
            day_housework = week_context_attributes[day][week_context_attributes[day].context == 'HouseWork']
            day_housework['period'] = day_housework['end'] - day_housework['start']
            c10b_housework_hours.append(day_housework['period'].sum())

    if np.min(c10a_exercising_hours) > 15 * 60:
        stress_score -=0.5

    if np.min(c10b_housework_hours) > 60 * 60:
        stress_score -= 0.5



    # condition 11a: Continuous 30 mins exercise, five times a week: -1 stress score
    # condition 11b: 15 mins exercise twice, five times a week: -1 stress score
    # condition 11c: 10 mins exercise thrice, five times a week: -1 stress score

    c11_day_cond = []
    for day in week_context_attributes.keys():
        if 'Exercising' not in week_context_attributes[day].context.values:
            c11_day_cond.append(False)
        else:
            day_exercise_values = week_context_attributes[day][week_context_attributes[day].context == 'OfficeWork']
            day_exercise_values['period'] = day_exercise_values['end'] - day_exercise_values['start']
            day_num_exercises = day_exercise_values.shape[0]
            day_min_continuous_exercise = day_exercise_values['period'].min()
            if (day_num_exercises >=1) and (day_min_continuous_exercise > 30*60):
                c11_day_cond.append(True)
            elif (day_num_exercises >=2) and (day_min_continuous_exercise > 15*60):
                c11_day_cond.append(True)
            elif (day_num_exercises >=3) and (day_min_continuous_exercise > 10*60):
                c11_day_cond.append(True)
            else:
                c11_day_cond.append(False)

    if (np.sum(c11_day_cond)/len(c11_day_cond)) > 0.7: # 5+ days in a week
        stress_score -= 1

    # condition 12: Working hours more than 8 for more than 5 days: +1 stress score
    c12_work_hours = c8_work_hours

    if np.percentile(c12_work_hours,0.7) > (8 * 60 * 60):
        stress_score += 1

    # condition 13a: Inactivity/Amusement for more than 2 hours everyday: -0.5 stress score
    # condition 13b: Inactivity/Amusement/Housework for more than 5 hours atleast two days: -0.5 stress score
    c13a_inactivity_amusement_hours = []
    for day in week_context_attributes:
        day_13a_hours = week_context_attributes[day][week_context_attributes[day].context.isin(['Inactivity','Amusement'])]
        if day_13a_hours.shape[0] > 0.:
            c13a_inactivity_amusement_hours.append((day_13a_hours['end'] - day_13a_hours['start']).sum())
        else:
            c13a_inactivity_amusement_hours.append(0.)

    c13b_inactivity_amusement_housework_hours = []
    for day in week_context_attributes:
        day_13b_hours = week_context_attributes[day][week_context_attributes[day].context.isin(['Inactivity','Amusement','HouseWork'])]
        if day_13b_hours.shape[0] > 0.:
            c13b_inactivity_amusement_housework_hours.append((day_13b_hours['end'] - day_13b_hours['start']).sum())
        else:
            c13b_inactivity_amusement_housework_hours.append(0.)

    if np.min(c13a_inactivity_amusement_hours) > (2 * 60 * 60):
        stress_score -= 0.5
    if np.percentile(c13b_inactivity_amusement_housework_hours, 30) > (5 * 60 * 60):
        stress_score -= 0.5

    # condition 14a: Working hours between 12am and 8am for any day: +1 stress score
    # condition 14b: Working hours between 12am and 8am for more than 4 days: +1 stress score

    c14_working_periods = c5_working_periods
    c14_night_work = []
    for day in c14_working_periods.keys():
        if np.sum(c14_working_periods[day][:8]) > 1:
            c14_night_work.append(True)
        else:
            c14_night_work.append(False)

    if np.sum(c14_night_work) >=1:
        stress_score +=1
    if np.sum(c14_night_work) >=4:
        stress_score += 1

    return stress_score


In [2]:
df_onto_casas = pd.read_csv("../../cache/trace_results/casas_onto_trace.csv")
df_onto_casas['tao_prediction'] = df_onto_casas['tao_prediction'].apply(lambda x: x.split(";") if not (str(x)=='nan') else ['Unknown'])
df_onto_casas.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../cache/trace_results/casas_onto_trace.csv'

In [ ]:
ts_dict = pickle.load(open("../../cache/trace_results/ts_results_casas.pb","rb"))
ts_results, cluster_labels = ts_dict['ts_results'], ts_dict['clusters']

In [47]:
wellness_user_inputs = {}

In [113]:

id = 'csh115'

In [114]:

print(f"Start id {id}")
df_ts_input =ts_results[id]
df_ts_input.info()
pred_min_ts, pred_max_ts = df_ts_input.start_timestamp.min(), df_ts_input.end_timestamp.max()
df_pred_ts = pd.DataFrame(np.arange(pred_min_ts, pred_max_ts + 1), columns=['timestamp'])
df_pred_ts['pred_context'] = None
df_pred_ts = df_pred_ts.set_index('timestamp')
for row_idx, row in df_ts_input.iterrows():
    df_pred_ts.loc[row['start_timestamp']:row['end_timestamp'], 'pred_context'] = df_pred_ts.loc[
                                                                                  row['start_timestamp']:
                                                                                  row['end_timestamp'],
                                                                                  'pred_context'].apply(
        lambda x: (x + cluster_labels[int(row['cluster_id'])]) if (x is not None) else cluster_labels[
            int(row['cluster_id'])])
df_pred_ts = df_pred_ts.reset_index().rename(columns={'pred_context':'context'})
df_pred_ts = df_pred_ts[~df_pred_ts.context.isnull()]
df_pred_ts.head()

Start id csh115
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205477 entries, 0 to 205476
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   start_timestamp       205477 non-null  float64
 1   next_start_timestamp  205477 non-null  float64
 2   end_timestamp         205477 non-null  float64
 3   cluster_id            205477 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 6.3 MB


,timestamp,context
0,1.308097e+09,[Sleeping]
1,1.308097e+09,[Sleeping]
2,1.308097e+09,[Sleeping]
3,1.308097e+09,[Sleeping]
4,1.308097e+09,[Sleeping]


In [115]:
df_onto_ts = df_onto_casas[df_onto_casas.id==id][['timestamp','tao_prediction']].rename(columns={'tao_prediction':'context'})
df_onto_ts = df_onto_ts[~df_onto_ts.context.isnull()]
df_onto_ts.head()

,timestamp,context
3837146,1308096677,[Sleeping]
3837147,1308096679,[Sleeping]
3837148,1308096682,[Sleeping]
3837149,1308096683,[Sleeping]
3837150,1308096854,[Sleeping]


In [116]:
df_combined_ts = pd.merge(df_onto_ts, df_pred_ts, on=['timestamp'], suffixes=('_onto','_tp'))
df_combined_ts.head()
df_combined_ts['context'] = df_combined_ts.apply(lambda row: ",".join(list(row['context_tp'])+list(row['context_onto'])),axis=1)
df_combined_ts = df_combined_ts[['timestamp','context']]
all_contexts = ['Exercising','HouseWork','Commuting','Sleeping','OfficeWork']
for context in all_contexts:
    df_combined_ts[context] = df_combined_ts['context'].apply(lambda x: context in x)
df_combined_ts = df_combined_ts.drop('context',axis=1)
df_combined_ts = pd.melt(df_combined_ts, id_vars='timestamp',var_name='context',value_name='isPresent')
df_combined_ts = df_combined_ts[df_combined_ts.isPresent==True][['timestamp','context']]
wellness_user_inputs[id] =df_combined_ts
print(f"Done id {id}")


Done id csh115


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40442 entries, 0 to 40441
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   start_timestamp       40442 non-null  float64
 1   next_start_timestamp  40442 non-null  float64
 2   end_timestamp         40442 non-null  float64
 3   cluster_id            40442 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 1.2 MB


,pred_context,context
timestamp,,
1.342781e+09,None,[Sleeping]
1.342781e+09,None,[Sleeping]
1.342781e+09,None,[Sleeping]
1.342781e+09,None,[Sleeping]
1.342781e+09,None,[Sleeping]


In [117]:
user_wellness_scores = {}
for id in wellness_user_inputs.keys():
    df_wellness_input = wellness_user_inputs[id].copy(deep=True)
    df_wellness_input['datetime'] = pd.to_datetime(df_wellness_input['timestamp'], unit='s')
    df_wellness_input['week'] = df_wellness_input['datetime'].apply(lambda x: x.strftime("%Y_%V"))
    df_wellness_input['day'] = df_wellness_input['datetime'].apply(lambda x: x.strftime("%Y-%m-%d"))

    user_week_context_attributes = []
    for week in df_wellness_input['week'].unique():
        df_week_wellness_input = df_wellness_input[df_wellness_input.week == week]
        week_context_attributes = {}
        for day in df_week_wellness_input['day'].unique():
            df_day_wellness_input = df_week_wellness_input[df_week_wellness_input.day == day]
            df_day_wellness_input['context_grp'] = (
                    df_day_wellness_input['context'] != df_day_wellness_input['context'].shift(1)).cumsum()
            day_contexts = df_day_wellness_input.groupby(['context_grp', 'context'], as_index=False).agg({
                'timestamp': ['min', 'max', lambda x: x.max() - x.min()]
            })
            day_contexts.columns = ['group', 'context', 'start', 'end', 'length']
            week_context_attributes[day] = day_contexts
        user_week_context_attributes.append([week, week_context_attributes])
        print(f'Done week {week}.')
    weekly_stress_scores = []
    for week, week_context_attributes in user_week_context_attributes:
        week_stress_score = get_weekly_stress_score(week_context_attributes)
        weekly_stress_scores.append((week, week_stress_score))
    user_wellness_scores[id] = weekly_stress_scores
    print(f"Done user {id}")

Done week 2012_29.
Done week 2012_30.
Done week 2012_31.
Done week 2012_32.
Done week 2012_33.
Done week 2012_34.
Done week 2012_35.
Done week 2012_36.
Done week 2012_37.
Done week 2012_38.
Done user csh101
Done week 2011_24.
Done week 2011_25.
Done week 2011_26.
Done week 2011_27.
Done week 2011_28.
Done week 2011_29.
Done week 2011_30.
Done week 2011_31.
Done week 2011_32.
Done week 2011_33.
Done user csh102
Done week 2011_24.
Done week 2011_25.
Done week 2011_26.
Done week 2011_27.
Done week 2011_28.
Done week 2011_29.
Done week 2011_30.
Done week 2011_31.
Done week 2011_32.
Done user csh103
Done week 2011_24.
Done week 2011_25.
Done week 2011_26.
Done week 2011_27.
Done week 2011_28.
Done week 2011_29.
Done week 2011_30.
Done week 2011_31.
Done week 2011_32.
Done user csh104
Done week 2011_24.
Done week 2011_25.
Done week 2011_28.
Done week 2011_29.
Done week 2011_30.
Done week 2011_31.
Done week 2011_32.
Done week 2011_26.
Done user csh105
Done week 2011_24.
Done week 2011_25.
Don

In [127]:
df_stress =None
for id in user_wellness_scores.keys():
    df_user_stress = pd.DataFrame(user_wellness_scores[id],columns=['week','score'])
    df_user_stress['id'] = id
    df_user_stress = df_user_stress[['id','week','score']].sort_values(by='week').reset_index()
    df_user_stress.columns =['week_id','id','week','score']
    df_user_stress['week_id'] = df_user_stress['week_id'].apply(lambda x: 'week '+ str(x+1))
    df_user_stress = df_user_stress.iloc[:5,:]
    if df_stress is None:
        df_stress = df_user_stress.copy(deep=True)
    else:
        df_stress = pd.concat([df_stress, df_user_stress],ignore_index=True)


In [128]:
df_stress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   week_id  75 non-null     object 
 1   id       75 non-null     object 
 2   week     75 non-null     object 
 3   score    75 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.5+ KB


In [129]:
df_stress.to_csv("../../cache/stress_scores.csv",index=False)

In [33]:
df_wellness_input = df_combined_ts

In [34]:
df_wellness_input['datetime'] = pd.to_datetime(df_wellness_input['timestamp'], unit='s')
df_wellness_input['week'] = df_wellness_input['datetime'].apply(lambda x: x.strftime("%Y_%V"))
df_wellness_input['day'] = df_wellness_input['datetime'].apply(lambda x: x.strftime("%Y-%m-%d"))


In [35]:
user_week_context_attributes = []
for week in df_wellness_input['week'].unique():
    df_week_wellness_input = df_wellness_input[df_wellness_input.week == week]
    week_context_attributes = {}
    for day in df_week_wellness_input['day'].unique():
        df_day_wellness_input = df_week_wellness_input[df_week_wellness_input.day == day]
        df_day_wellness_input['context_grp'] = (
                df_day_wellness_input['context'] != df_day_wellness_input['context'].shift(1)).cumsum()
        day_contexts = df_day_wellness_input.groupby(['context_grp', 'context'], as_index=False).agg({
            'timestamp': ['min', 'max', lambda x: x.max() - x.min()]
        })
        day_contexts.columns = ['group', 'context', 'start', 'end', 'length']
        week_context_attributes[day] = day_contexts
    user_week_context_attributes.append([week, week_context_attributes])
    print(f'Done week {week}.')

Done week 2011_24.
Done week 2011_25.
Done week 2011_26.
Done week 2011_27.
Done week 2011_28.
Done week 2011_29.
Done week 2011_30.
Done week 2011_31.
Done week 2011_32.
Done week 2011_33.


In [36]:
weekly_stress_scores = []
for week, week_context_attributes in user_week_context_attributes:
    week_stress_score = get_weekly_stress_score(user_week_context_attributes[0][1])
    weekly_stress_scores.append((week, week_stress_score))
weekly_stress_scores

[('2011_24', 1.0),
 ('2011_25', 1.0),
 ('2011_26', 1.0),
 ('2011_27', 1.0),
 ('2011_28', 1.0),
 ('2011_29', 1.0),
 ('2011_30', 1.0),
 ('2011_31', 1.0),
 ('2011_32', 1.0),
 ('2011_33', 1.0)]

In [12]:
user_week_context_attributes[0]

['2012_29',
 {'2012-07-20':     group        context       start         end  length
  0       1       GoingOut  1342780734  1342781454     720
  1       3  UsingBathroom  1342782560  1342782599      39
  2       5  UsingBathroom  1342782609  1342782609       0
  3       6      PhoneCall  1342785004  1342785340     336
  4       9  UsingBathroom  1342785645  1342785930     285
  5      12       GoingOut  1342786003  1342786018      15
  6      13       ComingIn  1342786433  1342786450      17
  7      14       Sleeping  1342786485  1342787744    1259
  8      16       Sleeping  1342787819  1342789329    1510
  9      17        Unknown  1342789491  1342789491       0
  10     18       GoingOut  1342789494  1342789509      15
  11     19       ComingIn  1342795232  1342795247      15
  12     22  UsingBathroom  1342795272  1342795407     135
  13     25       Sleeping  1342795479  1342796390     911
  14     26        Unknown  1342796446  1342796450       4
  15     27     HavingMeal  13